In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain.globals import set_debug

In [2]:
MCP_SERVER_URL="http://localhost:8000/sse"
OLLAMA_URL="http://localhost:11434"

In [3]:
from IPython.core.magic import register_cell_magic
from IPython.core import magic_arguments

@register_cell_magic
def skip(line, cell):
    """Skip execution of the entire cell (even magics and ! commands)."""
    print("⏭️ Skipped cell.")
    # Do NOT call exec, run_cell, or anything else
    return None

In [4]:
!echo should see aip.db in directory
!ls ../../db/
!echo  "..."
!echo removing aip.db in directory
!rm ../../db/aip.db
!echo should NOT see aip.db in directory
!ls ../../db/
!echo  "..."
!echo Rebuilding database
!sqlite3 ../../db/aip.db < ../../db/init.sql
!echo  "..."
!echo should see aip.db in directory
!ls ../../db/

should see aip.db in directory
aip.db   init.sql
...
removing aip.db in directory
should NOT see aip.db in directory
init.sql
...
Rebuilding database
...
should see aip.db in directory
aip.db   init.sql


In [5]:
###############################################
# set_debug(True) to see agent processing details

set_debug(False)

In [6]:
client = MultiServerMCPClient(
    {
        "Demo": {
            "url": MCP_SERVER_URL,
            "transport": "sse",
        }
    }
)

In [7]:
tools = await client.get_tools()

print('MCP Tools:\n')
print('-----------\n')
for t in tools: 
    print(f'Tool name: {t.name}')
    print(t.description)
    print('..\n')

MCP Tools:

-----------

Tool name: cancel_booked_appointment_for_client

Cancels a client appointment with a provider at a specific slot number.

RULES:
- Inputs (client_id provider_name and slot_number) may come from free-text user input.
- Ensure that the database is updated
..

Tool name: get_provider_names
returns a list of available providers
..

Tool name: get_provider_roles
returns a list of available provider roles
..

Tool name: get_available_booking_slots_for_provider

Returns booking 5 earliest time slots for a provider.

Output is ALWAYS JSON with fields:
  - slot_number (integer, sorted ascending)
  - provider_name (string)
  - time_slot (string, format 'YYYY-MM-DD HH24:MI:SS')

..

Tool name: book_slot_for_provider

Books a client appointment with a provider at a specific slot number.

RULES:
- Inputs (provider_name and slot_number) may come from free-text user input.
- Only book the slot if the inputs exactly match a valid slot returned by
  get_available_booking_slots_

In [8]:
prompts = await client.get_prompt('Demo', 'configure_assistant')

print('MCP Prompts:\n')
for p in prompts:
    print(p.content)

MCP Prompts:


        You are a scheduling assistant. You must ONLY interact with the environment using the following tools:
        - list_booked_appointments_for_client → returns a list booked appointments for a client id
        - cancel_booked_appointment_for_client → cancels booked appointment for a specific client_id, provider_name and slot_number combination
        - get_provider_names → returns a list of available providers
        - get_provider_roles → returns a list of provider roles
        - get_available_booking_slots_for_provider → returns list of 5 earliest available time slots for a provider. 
          response is JSON with a top-level "result" field.
          "result" is a list of dicts, each with:
            - slot_number (integer, sorted ascending)
            - provider_name (string)
            - time_slot (string in 'YYYY-MM-DD HH24:MI:SS' format).

        - book_slot_for_provider → books a client appointment with a provider at a specified slot number.

   

In [9]:
###############################################
# LLM to use in react agent

llm = ChatOllama(model="qwen3:8b", base_url=OLLAMA_URL, reasoning=True, temperature=0)

In [10]:
###############################################
# react agent creation

agent = create_react_agent(
    llm,
    tools, 
    prompt=prompts[0].content
)

In [11]:
#####################################################
# List of providers at facility

resp = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "What care providers are available in our faciliy and what are their roles?"}]}
)
print(resp['messages'][-1].content)

Here are the care providers available in the facility along with their roles:

1. **Susan Davis** - Registered Nurse  
2. **Lisa Brown** - Occupational Therapist  
3. **Daniel Miller** - Geriatrician  
4. **Emily Johnson** - Social Worker  
5. **Jane Smith** - Nutritionist  
6. **John Doe** - Physical Therapist  
7. **Kevin Anderson** - General Practitioner  
8. **Michael Green** - Pharmacist  
9. **Robert Martinez** - Care Coordinator  
10. **Sarah Wilson** - Home Health Aide  

Let me know if you need further details!


In [12]:
#####################################################
# List of open booking slots for provider (Susan Davis) at facility

user_message = {
    "role": "user",
    "content": "I want to book an apointment with Susan Davis ? Find the 5 earliest times."
}

resp = await agent.ainvoke(
    {"messages": resp['messages'] + [user_message]}
)

# Print the agent's last message
print(resp['messages'][-1].content)

Here are the 5 earliest available time slots for **Susan Davis**:

1. **Slot 44** - 2024-08-07 11:30:00  
2. **Slot 45** - 2024-08-07 12:00:00  
3. **Slot 46** - 2024-08-07 12:30:00  
4. **Slot 47** - 2024-08-07 13:00:00  
5. **Slot 48** - 2024-08-07 13:30:00  

Would you like to book one of these slots? If so, please specify the **slot number** and provide your **client ID**.


In [13]:
#####################################################
# List bookings for client id 123 - (Should not have any appoinments)

user_message = {
    "role": "user",
    "content": "What are the booked slots for client id 123?"
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

The client with ID 123 has no booked appointments.


In [14]:
#####################################################
# Book appointment with Susan Davis for client id 123 at time slot 44

user_message = {
    "role": "user",
    "content": "Book an appointment with Susan Davis at time slot_number slot 44 for client id 123"
}

resp = await agent.ainvoke(
    {
        "messages": resp['messages'] + [user_message]
    }
)

print(resp['messages'][-1].content)

The appointment has been successfully booked for client ID 123 with Susan Davis at slot number 44. The time slot is 2024-08-07 11:30:00.


In [15]:
#####################################################
# List bookings for client id 123 - (Should have 1 appoinment)

user_message = {
    "role": "user",
    "content": "What are the booked slots for client id 123?"
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

The booked slots for client id 123 are:

- **Provider:** susan davis  
- **Slot Number:** 44  
- **Time Slot:** 2024-08-07 11:30:00


In [16]:
#####################################################
# List of open booking slots for provider at facility - (The slot booked 44 should no longer be in this list)

user_message = {
    "role": "user",
    "content": "I want to schedule an apointment with Susan Davis ? Find the 5 earliest times."
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

Here are the 5 earliest available time slots for Susan Davis:

1. Slot #45 - 2024-08-07 12:00:00  
2. Slot #46 - 2024-08-07 12:30:00  
3. Slot #47 - 2024-08-07 13:00:00  
4. Slot #48 - 2024-08-07 13:30:00  
5. Slot #49 - 2024-08-07 14:00:00  

Please provide your client ID and select a slot number to book.


In [17]:
#####################################################
# Cancel booking for client id 123 with susan davis at slot 44

user_message = {
    "role": "user",
    "content": "Cancel appointment for client id 123 with susan davis at slot 44"
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

The appointment for client id 123 with Susan Davis at slot 44 has been successfully cancelled.


In [18]:
#####################################################
# List bookings for client id 123 - (Should not have any appoinments)

user_message = {
    "role": "user",
    "content": "What are the booked slots for client id 123?"
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

The client with ID 123 has no booked appointments.


In [19]:
#####################################################
# List of open booking slots for provider at facility - (The slot booked 44 be in this list again)

user_message = {
    "role": "user",
    "content": "I want to schedule an apointment with Susan Davis ? Find the 5 earliest times."
}

resp = await agent.ainvoke(
    {
        "messages": [user_message]
    }
)

# Print the agent's last message
print(resp['messages'][-1].content)

The 5 earliest available time slots for Susan Davis are:
1. 2024-08-07 11:30:00 (slot_number 44)
2. 2024-08-07 12:00:00 (slot_number 45)
3. 2024-08-07 12:30:00 (slot_number 46)
4. 2024-08-07 13:00:00 (slot_number 47)
5. 2024-08-07 13:30:00 (slot_number 48)


In [20]:
##########################################################################################################
## For Hung JR.. Demo... List alocated provider team members availability
##########################################################################################################

In [21]:
#####################################################
# List of open booking slots for a list of providers - Example call for Hung Jr.
user_message = {
    "role": "user",
    "content": "I want to see booking availabilities for these providers : Susan Davis, Lisa Brown and Jane Smith."
}

resp = await agent.ainvoke(
    {"messages": [user_message]}
)

# Print the agent's last message
print(resp['messages'][-1].content)

Here are the available booking slots for the requested providers:

**Susan Davis**  
- Slot 44: 2024-08-07 11:30:00  
- Slot 45: 2024-08-07 12:00:00  
- Slot 46: 2024-08-07 12:30:00  
- Slot 47: 2024-08-07 13:00:00  
- Slot 48: 2024-08-07 13:30:00  

**Lisa Brown**  
- Slot 37: 2024-08-07 08:00:00  
- Slot 38: 2024-08-07 08:30:00  
- Slot 40: 2024-08-07 09:30:00  
- Slot 42: 2024-08-07 10:30:00  
- Slot 43: 2024-08-07 11:00:00  

**Jane Smith**  
- Slot 37: 2024-08-07 08:00:00  
- Slot 38: 2024-08-07 08:30:00  
- Slot 39: 2024-08-07 09:00:00  
- Slot 40: 2024-08-07 09:30:00  
- Slot 41: 2024-08-07 10:00:00  

Let me know if you need to book or cancel any slots.
